In [1]:
import numpy as np
import pandas as pd

from astropy.io import fits

#fits_image_filename = fits.util.get_testdata_filepath()

In [2]:
_epsilon = 1e-2

In [3]:
hdul_gev = fits.open('data/gll_psc_v16.fit')
cat_gev = hdul_gev[1].data

names_gev = [
    'CLASS1',    
    'RAJ2000',
    'DEJ2000', 
    'GLON', 
    'GLAT', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000',  
    'Flux30_100',
    ] 


In [4]:
hdul_tev = fits.open('data/gammacat.fits.gz')
cat_tev = hdul_tev[1].data
names_tev = [
    'classes', 
    'glat', 
    'glon', 
    'morph_pa', 
    'pos_ra',
    'pos_dec',
    'sed_dnde', 
    'sed_dnde_err', 
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err'
    ]

In [5]:
_names_common = [
    'glat',
    'glon',
    'morph_pa',
    'pos_ra',
    'pos_dec',
    'sed_dnde',
    'sed_dnde_err',
    'sed_e_ref', 
    'spec_dnde_1TeV', 
    'spec_dnde_1TeV_err', 
    'spec_eflux_1TeV_10TeV', 
    'spec_eflux_1TeV_10TeV_err', 
    'spec_flux_1TeV', 
    'spec_flux_1TeV_crab', 
    'spec_flux_1TeV_crab_err', 
    'ASSOC_TEV', 
    'Variability_Index', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'Flux1000', 
    'Flux10000_100000', 
    'Flux1000_3000', 
    'Flux100_300', 
    'Flux3000_10000', 
    'Flux300_1000', 
    'Flux30_100', 
    'CLASS1']

In [6]:
def corresponding_bins(cat_gev, cat_tev, epsilon):
    d = {}
    
    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for j in range(len(glat_tev)):
        for i in range(len(glat_gev)):
            if ((class_tev[j].find('bin') != -1) 
                & ((class_gev[i].find('BIN') != -1) | (class_gev[i].find('HMB') != -1))
                & (np.abs(glat_gev[i] - glat_tev[j]) < epsilon) 
                & (np.abs(glon_gev[i] - glon_tev[j]) < epsilon)):
                d.update({j : i})
    return d
                
print(corresponding_bins(cat_gev, cat_tev, 1))

{13: 319, 45: 1172, 125: 2358}


In [7]:
gevToTev = {'BLL': 'blazar', 
            'FRSQ': 'frsq', 
            'HMB': 'bin' , 
            'BIN': 'bin', 
            'GAL': 'galaxy', 
            'PSR': 'psr', 
            'PWN': 'pwn', 
            'SNR': 'snr', 
            '': 'unid'}
tevToGev = {v:k for k, v in gevToTev.items()}

In [8]:
def common(cat_gev, cat_tev, epsilon):
    """This function looks for the same objects in GeV and TeV catalogs
    
    Return: a dictionary with corresponding values; ; a dictionary with corresponding classes
    
    cat_gev - table of GeV 
    cat_tev - table of TeV 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    d_classes = {}

    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for i in range(len(glat_gev)):
        for j in range(len(glat_tev)):
            start = 2
            #classGeV = class_gev[i]
            #start = -1
            #try: 
            #    start = (class_tev[j].find(gevToTev[classGeV]))
            #except KeyError:
            #    continue
            if (start != -1):
                if ((np.abs(glat_gev[i] - glat_tev[j])/np.abs(glat_gev[i]) < epsilon) and (np.abs(glon_gev[i] - glon_tev[j])//np.abs(glat_tev[j]) < epsilon)) :
                    try:
                        list(d.keys()).index(j)
                        d[j].append(i)
                    except ValueError:
                        d.update({j : [i]})
                    try:
                        list(d_classes.keys()).index(class_tev[j])
                        d_classes[class_tev[j]].append(class_gev[i])
                    except ValueError:
                        d_classes.update({class_tev[j] : [class_gev[i]]})
    return d, d_classes

D, d_classes = common(cat_gev, cat_tev, _epsilon)
print(D)
print(d_classes)

{1: [33, 65, 3028, 3032], 165: [33, 3028, 3032], 7: [67, 214, 226, 307, 364], 3: [75, 160, 161], 4: [78], 23: [97, 594, 602, 611, 2886], 5: [99, 115], 6: [101, 193, 270], 9: [136, 273], 12: [176, 255, 304], 10: [219, 278], 11: [219, 278], 14: [233, 329, 359, 362, 367], 8: [241], 17: [305, 328, 376, 388, 430, 489], 13: [319], 18: [346, 377, 440], 19: [349, 420, 471, 476, 486], 15: [384], 16: [387], 20: [450, 510, 514, 532], 21: [562, 648], 22: [588, 635], 26: [609, 641, 2827], 27: [609, 626, 641, 646, 2827], 24: [619, 620, 621], 25: [619, 620, 621], 28: [651, 661, 666], 31: [725], 32: [752], 33: [757], 34: [787, 793], 35: [808, 858, 2192], 36: [905, 907, 913, 917, 927], 37: [955], 38: [984, 1038], 41: [1042, 1045, 1099, 1128, 1310, 1338, 1875], 51: [1079, 1080, 1082, 1102, 1209, 1373, 1466, 1857, 1900, 1938], 52: [1104, 1125, 1374, 1380, 1460], 40: [1119, 1229, 1371, 1443], 43: [1131, 1135, 1157, 1216, 1445, 1496], 42: [1146], 44: [1162], 48: [1195, 1293, 1337], 49: [1297, 1318, 1402, 1

In [9]:
def common_class(cat_gev, cat_tev, epsilon):
    """This function looks for the same objects in GeV and TeV catalogs
    
    Return: dictionary with corresponding values
    
    cat_gev - table of GeV 
    cat_tev - table of TeV 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}

    class_gev = cat_gev['CLASS1']
    class_tev = cat_tev['classes']
    glat_gev = cat_gev['GLAT']
    glat_tev = cat_tev['glat']
    glon_gev = cat_gev['GLON']
    glon_tev = cat_tev['glon']
    
    for i in range(len(glat_gev)):
        for j in range(len(glat_tev)):
            start = 2
            classGeV = class_gev[i]
            start = -1
            try: 
                start = (class_tev[j].find(gevToTev[classGeV]))
            except KeyError:
                continue
            if (start != -1):
                if ((np.abs(glat_gev[i] - glat_tev[j])/np.abs(glat_gev[i]) < epsilon) and (np.abs(glon_gev[i] - glon_tev[j])//np.abs(glat_tev[j]) < epsilon)) :
                    d.update({j : i})
                    print(str(class_gev[i]) + ' - ' + str(class_tev[j]) )
    return d

D = common_class(cat_gev, cat_tev, _epsilon)
print(D)

HMB - bin
PWN - pwn
PWN - pwn
PSR - psr
PSR - psr,pwn
PWN - pwn
PWN - pwn
PWN - pwn
HMB - bin
SNR - snr,mc
{13: 319, 25: 620, 24: 621, 31: 725, 37: 955, 61: 1567, 85: 1965, 125: 2358, 145: 2499}


In [10]:
gevToTev = {'BLL': 'blazar', 
            'FRSQ': 'frsq', 
            'HMB': 'bin' , 
            'BIN': 'bin', 
            'GAL': 'galaxy', 
            'PSR': 'psr', 
            'PWN': 'pwn', 
            'SNR': 'snr', 
            '': 'unid'}
tevToGev = {v:k for k, v in gevToTev.items()}

In [11]:
d_coord = {'GLON' : 'glon', 
            'GLAT' : 'glat', 
            'RAJ2000' : 'pos_ra', 
            'DEJ2000' : 'pos_dec'}

In [12]:
def create_common_data(cat_gev, cat_tev, D, namefinal):
    """The fonction adds objects found both in GeV and TeV.
    
    cat_gev - table of GeV 
    cat_tev - table of TeV 
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    namefinal(list) - names of required features to fill in data
    """
    data = []
    k = 0
    for j in D.keys():
        data.append([]) 
        for i in range(15): 
            data[k].append(cat_tev[namefinal[i]][j]) 
        for i in range(15, len(namefinal)): 
            data[k].append(cat_gev[namefinal[i]][D[j]])
        k = k+1  
    df_common = pd.DataFrame(data = data, columns = namefinal)
    return df_common 

In [13]:
def create_common_data2(cat_gev, cat_tev, D, namefinal):
    """The fonction adds objects found both in GeV and TeV.
    
    cat_gev - table of GeV 
    cat_tev - table of TeV 
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    namefinal(list) - names of required features to fill in data
    """
    cat_gev_common = [cat_gev.tolist()[i] for i in D.values()] 
    cat_tev_common = [cat_tev.tolist()[i] for i in D.keys()] 
    
    df_gev = pd.DataFrame(data = cat_gev_common, columns = [c.name for c in cat_gev.columns]) 
    df_tev = pd.DataFrame(data = cat_tev_common, columns = [c.name for c in cat_tev.columns])
    
    df_common = df_gev.join(df_tev)
    df_common = df_common.loc[:, namefinal]

    return df_common

import time
start_time = time.time()
common_data = create_common_data(cat_gev, cat_tev, D, _names_common) 
print("first common--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
common_data = create_common_data2(cat_gev, cat_tev, D, _names_common) 
print("join common--- %s seconds ---" % (time.time() - start_time))

first common--- 0.047002553939819336 seconds ---
join common--- 0.6270360946655273 seconds ---


In [14]:
def create_only_tev_data(cat_tev, D, name_tev):
    """The fonction adds objects found only in TeV.
    
    cat_tev - table of TeV 
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_tev(list) - names of TeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_tev)):
        if not(j in D.keys()):
            data.append([])
            for i in range(len(name_tev)):
                data[k].append(cat_tev[name_tev[i]][j])
            k = k+1
            
    df_only_tev = pd.DataFrame(data = data, columns = name_tev)
    df_only_tev = df_only_tev.rename(columns = {'classes' : 'CLASS1'})    
    return df_only_tev

In [15]:
def create_only_gev_data(cat_gev, D, names_gev):
    """The fonction adds objects found only in GeV.
    
    cat_gev - table of GeV  
    D(dictionary) - a dictionary with repeated objects from TeV and GeV
    name_gev(list) - names of GeV columns
    """
    k = 0
    data = []
    
    for j in range(len(cat_gev)):
        if not(j in D.values()):
            data.append([])
            for i in range(len(names_gev)):
                data[k].append(cat_gev[names_gev[i]][j])
            k = k+1
            
    df_only_gev = pd.DataFrame(data = data, columns = names_gev)
    df_only_gev = df_only_gev.rename(columns = d_coord)        
    return df_only_gev

In [16]:
data = []
common_data = create_common_data(cat_gev, cat_tev, D, _names_common)
print("Added common values = " + str(len(common_data)))
only_tev_data = create_only_tev_data(cat_tev, D, names_tev)
print("Added values which can be found only in grammacat.fits (TeV) = " + str(len(only_tev_data)))
only_gev_data = create_only_gev_data(cat_gev, D, names_gev)
print("Added values which can be found only in gll_psc_v16.fit (GeV) = " + str(len(only_gev_data)))

print("Size of grammacat.fits (TeV) = "  + str(len(cat_tev)))
print("Size of gll_psc_v16.fits (GeV) = "  + str(len(cat_gev)))

Added common values = 9
Added values which can be found only in grammacat.fits (TeV) = 157
Added values which can be found only in gll_psc_v16.fit (GeV) = 3025
Size of grammacat.fits (TeV) = 166
Size of gll_psc_v16.fits (GeV) = 3034


In [17]:
common_data.head()

,glat,glon,morph_pa,pos_ra,pos_dec,sed_dnde,sed_dnde_err,sed_e_ref,spec_dnde_1TeV,spec_dnde_1TeV_err,...,Flux300_1000,Flux30_100,Flux1000,Flux10000_100000,Flux1000_3000,Flux100_300,Flux3000_10000,Flux300_1000,Flux30_100,CLASS1
0,1.086135,135.675278,NaN,NaN,NaN,"[3.2234e-12, 9.27542e-13, 2.32469e-13, 1.6299e...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.598415, 0.878288, 1.28897, 1.89222, 2.7774,...",4.800000e-12,4.000000e-13,...,1.962727e-07,NaN,4.606472e-08,5.130483e-10,3.871007e-08,5.552702e-07,6.510779e-09,1.962727e-07,NaN,HMB
1,-5.784357,184.557465,NaN,NaN,NaN,"[1.81e-10, 7.27e-11, 3.12e-11, 1.22e-11, 4.6e-...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.519, 0.729, 1.06, 1.55, 2.26, 3.3, 4.89, 7....",3.506046e-11,7.454410e-13,...,7.236911e-13,NaN,4.905819e-12,5.099059e-14,2.593843e-10,2.520464e-07,1.133945e-10,7.236911e-13,NaN,PWN
2,-5.784366,184.557465,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,5.445407e-07,NaN,1.573132e-07,2.263189e-09,1.255338e-07,1.645888e-06,2.545524e-08,5.445407e-07,NaN,PSR
3,4.265813,195.133850,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,...,1.374273e-06,NaN,6.921597e-07,3.276670e-09,5.716907e-07,1.782121e-06,1.170842e-07,1.374273e-06,NaN,PSR
4,-3.106103,263.331940,41.0,128.75,-45.599998,"[1.055e-11, 1.304e-11, 9.211e-12, 8.515e-12, 5...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.7185, 0.8684, 1.051, 1.274, 1.546, 1.877, 2...",1.359352e-11,7.530708e-13,...,8.192600e-08,NaN,1.834233e-08,6.866011e-10,1.446274e-08,3.718011e-07,3.186851e-09,8.192600e-08,NaN,PWN


In [18]:
only_tev_data.head()

,CLASS1,glat,glon,morph_pa,pos_ra,pos_dec,sed_dnde,sed_dnde_err,sed_e_ref,spec_dnde_1TeV,spec_dnde_1TeV_err,spec_eflux_1TeV_10TeV,spec_eflux_1TeV_10TeV_err,spec_flux_1TeV,spec_flux_1TeV_crab,spec_flux_1TeV_crab_err
0,"pwn,snr",10.203682,119.580254,17.4,1.608333,72.983612,"[1.9e-12, 7.3e-13, 1.2e-13, 3.4e-14, 1.2e-14, ...","[9e-13, 1.7e-13, 4e-14, 1.2e-14, 5e-15, 2.5e-1...","[0.78, 1.39, 2.47, 4.39, 7.81, 13.89, nan, nan...",1.020254e-12,2.673885e-13,3.016235e-12,4.314187e-13,8.502113e-13,4.098503,0.624576
1,hbl,-78.086937,74.632011,NaN,3.466667,-18.891388,"[1.907e-12, 1.462e-13, 8.40099e-15, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.4452, 0.949698, 2.121, nan, nan, nan, nan, ...",1.175430e-13,4.592630e-14,1.291623e-13,8.729060e-14,4.897623e-14,0.236093,0.136923
2,snr,1.412930,120.092361,NaN,6.340000,64.129997,"[1.83e-12, 2.29e-13, 4.56e-14, 6.62e-15, 6.05e...","[6.48e-13, 9.1e-14, 2.13e-14, 4.9e-15, 2.44e-1...","[0.5242, 0.9323, 1.658, 2.948, 5.242, 9.4, nan...",2.200000e-13,5.000000e-14,3.370669e-13,1.334992e-13,1.145833e-13,0.552357,0.174237
3,hbl,-81.216103,94.174644,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hbl,-2.978120,120.975426,NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
only_gev_data.head()

,CLASS1,pos_ra,pos_dec,glon,glat,Variability_Index,Flux1000,Flux10000_100000,Flux1000_3000,Flux100_300,Flux3000_10000,Flux300_1000,Flux30_100
0,,0.0377,65.751701,117.693878,3.402958,40.753517,1.015962e-09,2.839629e-11,1.236649e-09,1.808316e-08,5.780520e-11,6.940733e-09,NaN
1,,0.0612,-37.648399,345.410522,-74.946754,44.845318,2.038660e-10,1.590734e-11,1.450903e-10,9.876651e-11,7.452389e-11,1.342487e-10,NaN
2,spp,0.2535,63.243999,117.293091,0.925701,53.047859,6.403683e-10,2.021964e-12,3.020877e-10,2.913030e-08,1.158661e-10,4.650789e-09,NaN
3,bll,0.3209,-7.815900,89.022202,-67.324249,49.738213,6.952190e-10,1.005633e-11,5.223298e-10,4.086941e-09,2.106892e-10,2.083574e-09,NaN
4,fsrq,0.3612,21.337900,107.665428,-40.047157,130.336334,2.944868e-10,1.035538e-14,3.565206e-10,1.523905e-08,8.413887e-15,3.639644e-09,NaN
